In [419]:
import numpy as np

PATTERN_TO_DEBUG = 0

def toDec(hexstr,BIT):
    msb4bits = hexstr[0]
    n = int(msb4bits, 16)
    if n >= 8:
        p = -1*pow(2,BIT-1)
        addend = int(str(n-8) + hexstr[1:], 16)
        return str( p + addend)
    else:
        return str(int(hexstr, 16))

def int2hex(num,bit_width):
    if num<0:
        num = (1 << bit_width) + num
    hex_string = hex(num)[2:]
    return '{:0>{width}}'.format(hex_string,width = (bit_width+3)//4)

NEGATE = {'1': '0', '0': '1'}
def negate(value):
    return ''.join(NEGATE[x] for x in value)

def twocomplement(n, size_in_bits):
    # print(n)
    number = int(n)
    if number < 0:
        return negate(bin(abs(number) - 1)[2:]).rjust(size_in_bits, '1')
    else:
        return bin(number)[2:].rjust(size_in_bits, '0')

NEGATE = {'1': '0', '0': '1'}
def negate(value):
    return ''.join(NEGATE[x] for x in value)

def reverseRow(data, index):

    cols = len(data[index])
    for i in range(cols // 2):
        temp = data[index][i]
        data[index][i] = data[index][cols - i - 1]
        data[index][cols - i - 1] = temp

    return data

# Rotate Matrix by 180 degrees
def rotateMatrix(data):

    rows = len(data)
    cols = len(data[0])

    if (rows % 2):
        # If N is odd reverse the middle
        # row in the matrix
        data = reverseRow(data, len(data) // 2)

        # Swap the value of matrix [i][j] with
        # [rows - i - 1][cols - j - 1] for half
        # the rows size.
        for i in range(rows // 2):
            for j in range(cols):
                temp = data[i][j]
                data[i][j] = data[rows - i - 1][cols - j - 1]
                data[rows - i - 1][cols - j - 1] = temp
        return data

def twocomplement(n, size_in_bits):
    # print(n)
    number = int(n)
    if number < 0:
        return negate(bin(abs(number) - 1)[2:]).rjust(size_in_bits, '1')
    else:
        return bin(number)[2:].rjust(size_in_bits, '0')


In [420]:
with open('input.txt', 'r') as file:
  file_in = file.readlines()

NUM = int(file_in[0])

# print(file_in)
for i in range(NUM):
  img_16 = []
  kernal_16 = []
  matrix_idx = []
  mode = []

  # MATRIX SIZE
  matrix_in_idx = int(file_in[1 + 65 * i + 0])

  if matrix_in_idx == 0:
     matrix_size = 8
  elif matrix_in_idx == 1:
     matrix_size = 16
  else:
     matrix_size = 32

  for j in range(16):
    img_in = np.array([int(toDec(val,8)) for val in file_in[1 + 65 * i + (1+j)].split()])
    img_16.append(np.reshape(img_in,(matrix_size,matrix_size)))

  for k in range(16):
    kernal_in = np.array([int(toDec(val,8)) for val in file_in[1 + 65 * i + (17+k)].split()])
    kernal_16.append(np.reshape(kernal_in,(5,5)))


  for idx in range(16):
    mode.append(int(file_in[1 + 65 * i + (33+idx*2)])) #0 2 4
    matrix_idx.append(np.array([int(val) for val in file_in[1 + 65 * i + (34+idx*2)].split()]))# 1 3 5

  if i == PATTERN_TO_DEBUG:
    break

In [421]:
mode[0]

0

In [422]:
matrix_idx[6]

array([13, 10])

In [423]:
img_16[15]

array([[ 2,  1, -3, -1,  1, -3,  3, -1],
       [ 2,  1, -3, -3,  0,  0, -3,  2],
       [ 3, -1, -2, -2,  3,  1,  0,  1],
       [-3, -1, -2,  0,  0, -2, -1, -2],
       [ 0,  2,  2, -1,  1, -3, -1,  3],
       [ 2, -1,  2,  1, -2, -2,  2, -3],
       [ 2, -1,  2,  2,  1,  0, -3,  0],
       [-2,  0,  2, -1,  1, -1,  3, -3]])

In [424]:
kernal_16[15]

array([[-1,  0, -3,  0, -3],
       [ 0,  3, -3,  2, -2],
       [ 2,  2,  1,  1,  1],
       [ 3, -3,  1, -3,  1],
       [-1, -1,  2,  0, -3]])

In [425]:
matrix_idx[0]

array([0, 0])

In [426]:
matrix_size

8

## Convolution

In [427]:
img = img_16[13]
kernal = kernal_16[9]

convoluted_results = np.zeros((matrix_size-4,matrix_size-4))
for i in range(matrix_size-4):
    for j in range(matrix_size-4):
        conv = 0
        for k in range(5):
            for l in range(5):
                conv += kernal[k,l] * img[i+k,j+l]

        convoluted_results[i,j] = conv

In [428]:
convoluted_results

array([[ 18.,  -1.,  -7.,  17.],
       [-14.,   4.,  -6.,  18.],
       [ 17.,  -3.,   5., -23.],
       [-38.,  14.,  -7.,  12.]])

## Maxpooling

In [429]:
convoluted_img_width = len(convoluted_results)

In [430]:
max_pooling_results = np.zeros((convoluted_img_width//2,convoluted_img_width//2))

for i in range(0,convoluted_img_width,2):
    for j in range(0,convoluted_img_width,2):
        max_pooling_results[i//2,j//2] = max(convoluted_results[i,j],convoluted_results[i+1,j],\
                                                convoluted_results[i+1,j+1],convoluted_results[i,j+1])

In [431]:
max_pooling_results

array([[18., 18.],
       [17., 12.]])

### Merged Convolution and max pooling

In [432]:
img_16[2]

array([[ 3,  0, -1, -1, -3,  3, -2,  0],
       [ 0,  3, -3,  1, -3,  3,  3,  0],
       [ 0,  3,  3,  2, -1,  2, -2, -1],
       [-3,  2,  2,  3, -2,  1, -1, -2],
       [ 3,  3,  2, -1,  3,  3,  0,  2],
       [-3, -3,  0, -2, -3, -1,  3, -3],
       [-2,  1,  3, -3,  3,  2, -1, -1],
       [ 0, -3,  1, -1,  3, -3,  2,  0]])

In [433]:
kernal_16[3]

array([[-1,  1, -2, -3,  1],
       [-1,  1,  1,  0,  1],
       [-3,  1,  3,  3,  0],
       [ 2, -1,  3, -3,  0],
       [-1, -2,  1, -2,  1]])

In [434]:
img = img_16[13]
kernal = kernal_16[9]

In [435]:
max_pooling_results = np.zeros((convoluted_img_width//2,convoluted_img_width//2))
for img_yptr in range(0,matrix_size-4,2):
    for img_xptr in range(0,matrix_size-4,2):
        # print(img_yptr)

        temp_max = -999999
        # 4 times convolution to quickly get values needed for max pooling
        for mp_window_y in range(2):
            for mp_window_x in range(2):
                conv = 0
                idx_x = img_xptr+mp_window_x
                idx_y = img_yptr+mp_window_y
                for k_yptr in range(5):
                    # Uses High level modeling for fast MAC, img_x_cord[x] and kernal_x_cord[x]
                    for k_xptr in range(5): # Unrolling the x_ptr
                        conv += kernal[k_yptr,k_xptr] * img[idx_y+k_yptr,idx_x+k_xptr]

                if conv>=67:
                    print(conv)
                    print(f"(y,x) = {img_yptr,img_xptr}")
                    print(f"Window_y:{mp_window_y}, Window_x:{mp_window_x}")

                if (mp_window_y == 0 and mp_window_x == 0) or conv > temp_max:
                    temp_max = conv

        max_pooling_results[img_yptr//2,img_xptr//2] = temp_max

In [436]:
max_pooling_results

array([[18., 18.],
       [17., 12.]])

In [437]:
max_pooling_results = np.zeros((convoluted_img_width//2,convoluted_img_width//2))
for img_yptr in range(0,matrix_size-4,2):
    for img_xptr in range(0,matrix_size-4,2):
        temp_max = -999999
        # 4 times convolution to quickly get values needed for max pooling
        for mp_window_y in range(2):
            for mp_window_x in range(2):
                conv = 0
                idx_y = img_yptr+mp_window_y
                idx_x = img_xptr+mp_window_x
                for k_yptr in range(5):
                    conv += kernal[k_yptr,0] * img[idx_y+k_yptr,idx_x+0]
                    conv += kernal[k_yptr,1] * img[idx_y+k_yptr,idx_x+1]
                    conv += kernal[k_yptr,2] * img[idx_y+k_yptr,idx_x+2]
                    conv += kernal[k_yptr,3] * img[idx_y+k_yptr,idx_x+3]
                    conv += kernal[k_yptr,4] * img[idx_y+k_yptr,idx_x+4]

                if (mp_window_y == 0 and mp_window_x == 0) or conv > temp_max:
                    temp_max = conv

        max_pooling_results[img_yptr//2,img_xptr//2] = temp_max

In [438]:
max_pooling_results

array([[18., 18.],
       [17., 12.]])

### Transposed convolution

In [439]:
trans_convoluted_results = np.zeros((matrix_size+4,matrix_size+4))
for i in range(matrix_size):
    for j in range(matrix_size):
        for k in range(5):
            for l in range(5):
                conv = kernal[k,l] * img[i,j]
                trans_convoluted_results[i+k,j+l] += conv

In [440]:
trans_convoluted_results

array([[  0.,   0.,  -2.,   4.,   5.,  -8.,  -5.,   3.,   3.,   0.,   0.,
          0.],
       [  0.,  -2.,   1.,   8.,  -8.,   4.,  -4., -13.,  10.,  -5.,  -5.,
         -1.],
       [  0.,  -2.,   0.,   2.,   1.,  -9., -21.,   9., -16.,  13.,  -6.,
         -3.],
       [  3.,  -6.,   2.,  21.,   6.,  -2.,   1.,  -3.,  14.,  -4.,  10.,
          2.],
       [  7.,   3.,  22.,   7.,  12.,   2.,  -2.,  15.,  -1.,  -7.,  13.,
          3.],
       [ 12.,   5.,   1., -28.,  10.,  -1., -13., -20.,   5.,   9.,  -7.,
          8.],
       [  5.,  -6.,  13.,   6.,  16.,   2.,   3., -29.,   2., -17.,   2.,
        -12.],
       [  0.,  10.,  21., -33., -13.,  -2., -17.,  18., -33.,  38.,   4.,
         -6.],
       [ -8.,   2.,  -9.,  13.,  18., -14.,  25.,  -1.,  12.,   3.,   3.,
          9.],
       [-13.,   8.,   6.,  14., -26.,   8., -24.,   7.,  -7.,  -7.,  -9.,
         -2.],
       [ -6.,  10., -16.,  11.,   2., -12.,   1., -13.,  12.,  12.,  -1.,
          0.],
       [  0.,   2.,  

## Transpoed convolution with zero-padding

In [441]:
img = img_16[13]

In [442]:
img

array([[ 0, -2,  2,  3, -3,  0,  0,  0],
       [ 0, -1,  2,  1,  1, -2,  3,  1],
       [ 3,  1,  3,  2,  1,  0, -1,  0],
       [ 1, -3, -3, -1,  2,  1, -3,  0],
       [ 1,  3,  3, -2,  0,  0, -2, -3],
       [ 0,  0, -3,  0, -2,  3, -2,  1],
       [-3,  3,  2, -1,  3, -2,  2,  3],
       [-2,  3, -3,  1,  0, -3, -2, -1]])

In [443]:
kernal = kernal_16[7]

In [444]:
kernal

array([[ 0,  2,  1, -3,  1],
       [ 2,  1, -1, -1, -2],
       [ 3, -1, -3,  0,  1],
       [-1, -1,  0, -1,  0],
       [ 3,  0,  2, -1, -1]])

In [445]:
new_trans_convoluted_results = np.zeros((matrix_size+4,matrix_size+4))
# Note the matrix is flipped
lower_bound0 = matrix_size + 4
lower_bound1 = matrix_size + 5
lower_bound2 = matrix_size + 6
lower_bound3 = matrix_size + 7

for img_ytr in range(matrix_size+4):
    for img_xptr in range(matrix_size+4):
        conv = 0
        for k_yptr in range(5):
            # Uses High level always@* , need idx_x[x]
            for k_xptr in range(5):
                idx_y = img_ytr + k_yptr
                idx_x = img_xptr + k_xptr
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or idx_y == 3 or idx_x == 0 or idx_x == 1 \
                    or idx_x == 2 or idx_x == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x == lower_bound0 or idx_x == lower_bound1 or idx_x == lower_bound2 or idx_x == lower_bound3 :
                        conv += 0
                else:
                   # Address translation
                   conv += kernal[4-k_yptr,4-k_xptr] * img[idx_y-4,idx_x-4]

        new_trans_convoluted_results[img_ytr,img_xptr] = conv


In [446]:
rotateMatrix(kernal)

array([[-1, -1,  2,  0,  3],
       [ 0, -1,  0, -1, -1],
       [ 1,  0, -3, -1,  3],
       [-2, -1, -1,  1,  2],
       [ 1, -3,  1,  2,  0]])

In [447]:
new_trans_convoluted_results

array([[  0.,   0.,  -4.,   2.,  14., -11., -10.,  12.,  -3.,   0.,   0.,
          0.],
       [  0.,  -4.,   0.,  13.,   4.,  -8.,  -8.,  -1.,  18., -10.,   0.,
          1.],
       [  0.,  -2.,  16.,  16.,  -9., -16.,   7.,   4.,  -5.,   3.,  -8.,
         -2.],
       [  6.,   6.,   6., -10.,  -3.,  -7.,   0.,  -8., -14.,   6.,   2.,
          1.],
       [ 11.,  -8.,   1.,   5., -11.,   5.,  -7., -12.,  -2.,   1.,  11.,
         -3.],
       [  2., -10.,   1.,  -9.,   8., -10., -19.,  -3.,   6.,  18.,  -5.,
          6.],
       [ 11.,   7.,  21.,   3., -12.,  18.,   5., -20.,  26.,   3.,  -5.,
         -2.],
       [ -4., -14.,  -8.,  -6.,  -2.,  20., -19.,   5.,  -1.,   5.,  -2.,
         -6.],
       [-10.,  25.,  25., -14.,  10., -21.,  -9.,   5., -10.,  -4.,  11.,
          8.],
       [ -3.,  11., -20.,  -1., -11.,   0., -12.,  12.,   1.,  -1.,  -4.,
         -2.],
       [ -7.,   8.,   0.,  10.,   9.,  -7.,  15.,   6.,   7.,   8.,  -4.,
         -3.],
       [ -6.,   9., -

In [448]:
np.array_equal(new_trans_convoluted_results,trans_convoluted_results)

False

In [449]:
new_trans_convoluted_results = np.zeros((matrix_size+4,matrix_size+4))
lower_bound0 = matrix_size + 4
lower_bound1 = matrix_size + 5
lower_bound2 = matrix_size + 6
lower_bound3 = matrix_size + 7

for img_ytr in range(matrix_size+4):
    for img_xptr in range(matrix_size+4):
        conv = 0
        if(img_yptr ==2 and img_xptr == 0): print("DEBUG")
        for k_yptr in range(5):
            # for k_xptr in range(5):
                idx_y = img_ytr + k_yptr
                idx_x_0 = img_xptr + 0
                idx_x_1 = img_xptr + 1
                idx_x_2 = img_xptr + 2
                idx_x_3 = img_xptr + 3
                idx_x_4 = img_xptr + 4
                # 0
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_0 == 0 or idx_x_0 == 1 \
                    or idx_x_0 == 2 or idx_x_0 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_0 == lower_bound0 or idx_x_0 == lower_bound1 or idx_x_0 == lower_bound2 or idx_x_0 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,4] * img[idx_y-4,idx_x_0-4]
                # 1
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_1 == 0 or idx_x_1 == 1 \
                    or idx_x_1 == 2 or idx_x_1 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_1 == lower_bound0 or idx_x_1 == lower_bound1 or idx_x_1 == lower_bound2 or idx_x_1 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,3] * img[idx_y-4,idx_x_1-4]
                # 2
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_2 == 0 or idx_x_2 == 1 \
                    or idx_x_2 == 2 or idx_x_2 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_2 == lower_bound0 or idx_x_2 == lower_bound1 or idx_x_2 == lower_bound2 or idx_x_2 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,2] * img[idx_y-4,idx_x_2-4]
                # 3
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_3 == 0 or idx_x_3 == 1 \
                    or idx_x_3 == 2 or idx_x_3 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_3 == lower_bound0 or idx_x_3 == lower_bound1 or idx_x_3 == lower_bound2 or idx_x_3 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,1] * img[idx_y-4,idx_x_3-4]

                # 4
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_4 == 0 or idx_x_4 == 1 \
                    or idx_x_4 == 2 or idx_x_4 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_4 == lower_bound0 or idx_x_4 == lower_bound1 or idx_x_4 == lower_bound2 or idx_x_4 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,0] * img[idx_y-4,idx_x_4-4]

        new_trans_convoluted_results[img_ytr,img_xptr] = conv


DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG


In [450]:
new_trans_convoluted_results

array([[  0.,   2.,   0.,  -9.,   4.,   3.,   0.,   9.,  -9.,   0.,   0.,
          0.],
       [  0.,   1.,   1.,  -7.,   1.,   3.,   2.,  -5.,  11.,  -4.,   9.,
          3.],
       [ -3.,  -6.,   5.,   4.,   5., -13.,  23.,  14.,  -8.,  -1.,  -7.,
         -1.],
       [ -1.,   2.,   7., -10., -15., -31.,   7.,  12., -11.,  -8.,   0.,
          3.],
       [  2.,  -4.,  -2.,  -5.,  -6.,  21.,  20.,   0.,  -2.,  -5.,   1.,
         -7.],
       [ -5., -10., -11.,  -5.,   9.,  -7., -11.,   1.,   5.,  25., -10.,
          6.],
       [  5.,   0.,   0.,   5., -17.,  16.,  21., -10.,  21.,   7.,  -2.,
         -1.],
       [  1., -11., -13.,  15.,  -5.,  11.,  -6.,   8.,  -6.,  -8., -25.,
         -9.],
       [ -2.,   5.,   9.,  -5.,   5.,   5.,  -4.,   2.,  16., -16.,  -3.,
         12.],
       [  4.,   0.,  -4.,  -3., -15.,  19., -27.,  12.,   9., -12.,   4.,
          3.],
       [  1.,   8.,  -5., -14.,  15.,   0.,  11.,  10., -11.,   0.,   1.,
         -2.],
       [ -2.,   9., -

#### Convert results to LSB binary representation

In [451]:
twocomplement(57,20)

'00000000000000111001'

In [452]:
vf = np.vectorize(twocomplement)
bits_result = vf(trans_convoluted_results,20)

In [453]:
bits_result[0][0]

'00000000000000000000'

In [454]:
result = bits_result[0][0]

In [455]:
result = result[::-1]

In [456]:
result

'00000000000000000000'